In [ ]:
from nbdev import *
%nbdev_default_export blocks

Cells will be exported to tf2_gans.blocks,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
from tensorflow.keras import layers, Model
import tensorflow as tf

# Blocks

In [ ]:
%nbdev_export
class UpsampleConv2D(Model):
    def __init__(self,
                 filters,
                 kernel_size,
                 upsample_interpolation='nearest',
                 upsample_size=(2, 2),
                 name='upsample_conv2d',
                 strides=(1, 1),
                 padding='same',
                 use_bias=False,
                 kernel_initializer='glorot_normal',
                 **kwargs):
        super(UpsampleConv2D, self).__init__(name=name, **kwargs)

        self.upsample = layers.UpSampling2D(size=upsample_size,
                                            interpolation=upsample_interpolation)
        self.conv = layers.Conv2D(filters,
                                  kernel_size,
                                  strides=strides,
                                  padding=padding,
                                  use_bias=use_bias,
                                  kernel_initializer=kernel_initializer)

    def call(self, inputs, training=False):
        x = self.upsample(inputs)
        return self.conv(x)

In [ ]:
%nbdev_export
class DownBlock(Model):

    def __init__(self,
                 filters,
                 kernel_size,
                 use_batch_norm=True,
                 name='down_block',
                 activation=layers.LeakyReLU,
                 strides=(2, 2),
                 padding='same',
                 use_bias=False,
                 kernel_initializer='glorot_normal',
                 **kwargs):
        super(DownBlock, self).__init__(name=name, **kwargs)

        self.conv = layers.Conv2D(filters,
                                  kernel_size,
                                  strides=strides,
                                  padding=padding,
                                  use_bias=use_bias,
                                  kernel_initializer=kernel_initializer)

        self.use_batch_norm = use_batch_norm
        if self.use_batch_norm:
            self.batch_norm = layers.BatchNormalization()

        self.activation = activation()

    def call(self, inputs, training=False):
        x = self.conv(inputs)
        if self.use_batch_norm:
            x = self.batch_norm(x)
        return self.activation(x)

In [ ]:
%nbdev_export
class UpBlock(Model):

    def __init__(self,
                 filters,
                 kernel_size,
                 use_upsample=True,
                 upsample_interpolation='nearest',
                 use_dropout=False,
                 dropout_rate=0.5,
                 name='up_block',
                 activation=layers.ReLU,
                 strides=(2, 2),
                 padding='same',
                 use_bias=False,
                 kernel_initializer='glorot_normal',
                 **kwargs):
        super(UpBlock, self).__init__(name=name, **kwargs)

        self.use_upsample = use_upsample
        if use_upsample:
            self.deconv = UpsampleConv2D(filters,
                                         kernel_size,
                                         upsample_interpolation=upsample_interpolation,
                                         padding=padding,
                                         use_bias=use_bias,
                                         kernel_initializer=kernel_initializer)
        else:
            self.deconv = layers.Conv2DTranspose(filters,
                                                 kernel_size,
                                                 strides=strides,
                                                 padding=padding,
                                                 use_bias=use_bias,
                                                 kernel_initializer=kernel_initializer)

        self.batch_norm = layers.BatchNormalization()

        self.use_dropout = use_dropout
        if self.use_dropout:
            self.dropout = layers.Dropout(dropout_rate)

        self.activation = activation()

    def call(self, inputs, training=False):
        x = self.deconv(inputs)
        if self.use_dropout:
            x = self.dropout(x)
        return self.activation(x)

In [ ]:
%nbdev_export
class ResBlock(Model):
    def __init__(self,
                 filters,
                 kernel_size=(3, 3),
                 strides=(1, 1),
                 padding_type="REFLECT",
                 use_dropout=False,
                 dropout_rate=0.5,
                 activation=layers.ReLU,
                 use_bias=False,
                 name='res_block',
                 padding='valid',
                 **kwargs):
        super(ResBlock, self).__init__()

        self.filters = filters
        self.padding_type = padding_type

        self.use_dropout = use_dropout
        if use_dropout:
            self.dropout = layers.Dropout(dropout_rate)

        self.conv1 = layers.Conv2D(filters=filters,
                                            kernel_size=kernel_size,
                                            strides=strides,
                                            padding=padding,
                                            use_bias=use_bias)
        self.conv2 = layers.Conv2D(filters=filters,
                                            kernel_size=kernel_size,
                                            strides=strides,
                                            padding=padding,
                                            use_bias=use_bias)

        self.batch_norm1 = layers.BatchNormalization()
        self.batch_norm2 = layers.BatchNormalization()

        self.activation = activation()

    def call(self, inputs, training=False):
        x = tf.pad(inputs,
                   [[0, 0], [1, 1], [1, 1], [0, 0]],
                   mode=self.padding_type,
                   constant_values=0)
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.activation(x)
        if self.use_dropout:
            x = self.dropout(x, training=training)
        x = tf.pad(x,
                   [[0, 0], [1, 1], [1, 1], [0, 0]],
                   mode=self.padding_type,
                   constant_values=0)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        return x + inputs

In [ ]:
notebook2script()

Converted 01_dcgan.ipynb.
Converted blocks.ipynb.
Converted core.ipynb.
Converted index.ipynb.
Converted losses.ipynb.
